# Exercises - Risk Metrics

# 1. Risk Metrics of a Stock

This problem uses data from `data/spx_returns_weekly.xlsx`.

Choose any stock to evaluate below.

In [368]:
import polars as pl
import math
from openpyxl import load_workbook
from pathlib import Path
from typing import List, Dict
import altair as alt

DATA_PATH = Path.cwd().parents[1] / "data"
FILE_PATH = DATA_PATH / "risk_etf_data.xlsx"
FREQ = 252

In [153]:
def print_sheetname(FILE_PATH: Path) -> List:  #@save
    wb = load_workbook(FILE_PATH, read_only=True)
    result = wb.sheetnames
    wb.close()
    return result

In [154]:
print_sheetname(FILE_PATH)

['descriptions', 'total returns', 'prices']

In [164]:
df = pl.read_excel(FILE_PATH, sheet_name="total returns")
df.tail(3)

Date,SPY,VEA,UPRO,GLD,USO,FXE,BTC,HYG,IEF,TIP,SHV
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2025-06-25,0.00056,-0.004987,0.001947,0.003037,0.004247,0.004011,0.012406,-0.000249,0.00042,0.000548,0.000091
2025-06-26,0.007824,0.010918,0.022657,-0.001107,0.003819,0.003995,-0.003737,0.00287,0.003568,0.00292,0.000181
2025-06-27,0.004968,0.006197,0.013495,-0.018124,-0.004212,0.000278,0.001201,-0.000498,-0.002614,-0.001274,0.000362


## 1.1 Return Moments

Report the moments of the returns. Annualize the mean and volatility.
* mean
* volatility
* skewness
* (excess) kurtosis

Note that the pandas function for kurtosis already reports **excess** kurtosis.

In [156]:
def select_cols(data: pl.DataFrame, tickers: List[str]) -> pl.DataFrame:    #@save
    return data.select(pl.col(tickers))

In [183]:
def calc_moment(data: pl.DataFrame, m: int) -> pl.DataFrame:    #@save
    """
    Calculate moment for return dataframe
    Args
        - data: rows as timeseries returns, columns as ticker names
        - m: moment (m >= 3)
    """
    return data.with_columns(
        (
            (pl.all() - pl.all().mean()).pow(m) / (pl.len() - 1)
        ).truediv(
            pl.all().std().pow(m)
        )
    ).sum()

In [171]:
def get_tickers(data: pl.DataFrame) -> List[str]:
    return data.select(pl.col(pl.Float64)).columns

In [184]:
tickers = get_tickers(df)
selected = select_cols(df, tickers=tickers)
annualized_mean = selected.mean() * FREQ
annualized_std = selected.std() * math.sqrt(FREQ)
skewness = calc_moment(selected, m=3)
kurtosis = calc_moment(selected, m=4)

In [185]:
annualized_mean

SPY,VEA,UPRO,GLD,USO,FXE,BTC,HYG,IEF,TIP,SHV
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.152921,0.097015,0.388461,0.128784,0.050176,0.015982,0.792938,0.046493,0.012586,0.027591,0.02158


In [168]:
annualized_std

SPY,VEA,UPRO,GLD,USO,FXE,BTC,HYG,IEF,TIP,SHV
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.188503,0.175062,0.564936,0.14276,0.38596,0.073682,0.698322,0.086814,0.068783,0.060184,0.002781


In [186]:
skewness

SPY,VEA,UPRO,GLD,USO,FXE,BTC,HYG,IEF,TIP,SHV
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.304253,-0.859636,-0.38059,-0.150257,-1.274386,0.16274,0.02712,0.129896,0.196439,0.362814,0.655011


In [188]:
kurtosis - 3

SPY,VEA,UPRO,GLD,USO,FXE,BTC,HYG,IEF,TIP,SHV
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
14.196294,16.070023,14.939922,2.687771,16.251028,1.341035,6.133385,26.344145,3.18301,14.810047,3.267674


## 1.2 Maximum Drawdown

Report the maximum drawdown for each return series.

If we resampled this data to weekly and recalculated the maximum drawdown, do you think it would be larger or smaller (in magnitude)?

In [351]:
def calc_max_dd(data: pl.DataFrame, ticker: str) -> Dict:   #@save
    """
    Calculate maximum drawdown for an asset
    Args:
        - data: asset returns, columns ['Date', 'tk1', 'tk2', ...]
        - ticker: one single ticker to evaluate
    Returns:
        - Dict:
            - max_drawdown: amount
            - peak: day that reach peak of the max drawdown
            - bottom: day that reach bottom of the max drawdown
            - recover: day recover, if no then "nan"
            - duration: time length to recover, if no then "nan"
    """
    # create new df, calculate cumulative return, peak, bottom
    df = (
        data.select([pl.col(pl.Date), ticker])
        .with_columns(
            (pl.col(ticker) + 1).cum_prod().alias("cum_return")
        )
        .with_columns(
            pl.col("cum_return").cum_max().alias("peak")
        )
        .with_columns(
            (pl.col("cum_return") / pl.col("peak") - 1).alias("drawdown")
        )
    )
    # filter the period where max drawdown is realized
    dd_period = df.filter(
        pl.col("drawdown").eq(pl.col("drawdown").min())
    )
    max_dd = dd_period["drawdown"][0]
    peak_value = dd_period["peak"]
    bottom_day = dd_period["Date"][0]

    peak_day = df.filter(
        (pl.col("cum_return").eq(peak_value)) & 
        (pl.col("Date").le(bottom_day))
    )["Date"][-1]  # Take the last occurrence before bottom

    recover_day = df.filter(
        pl.col("cum_return").ge(peak_value)
        & (pl.col("Date").gt(bottom_day))
    )[0]["Date"][0]
    
    result = {
        "max_drawdown": max_dd,
        "peak": peak_day,
        "bottom": bottom_day,
        "recover": recover_day,
        "duration": recover_day - peak_day,
        "drawdown": df.select(pl.col(["Date", "drawdown"]))
    }
    return result

In [393]:
def plot_line(
        data: pl.DataFrame, 
        x: str, 
        y: str|List[str]
    ) -> alt.Chart:
    """
    Time series line plot 
    Args
        - data: columns ["Date", "var1", "var2", ...]
        - x: Column name for x
        - y: Column name or name list for y
    """
    df_long = data.unpivot(
        on=y, index=x, variable_name="Variable", value_name="Value"
    )
    chart = alt.Chart(df_long).mark_line().encode(
        x=f'{x}:T',
        y='Value:Q',
        color='Variable:N'
    ).properties(width=500)
    return chart

In [394]:
dd_result = calc_max_dd(df, "SPY")
dd_series = dd_result["drawdown"]
plot_line(dd_series, x="Date", y=["drawdown"])

alt.Chart(...)

In [390]:
dd_result

{'max_drawdown': -0.33717268565971437,
 'peak': datetime.date(2020, 2, 19),
 'bottom': datetime.date(2020, 3, 23),
 'recover': datetime.date(2020, 8, 10),
 'duration': datetime.timedelta(days=173),
 'drawdown': shape: (2_132, 2)
 ┌────────────┬───────────┐
 │ Date       ┆ drawdown  │
 │ ---        ┆ ---       │
 │ date       ┆ f64       │
 ╞════════════╪═══════════╡
 │ 2017-01-04 ┆ 0.0       │
 │ 2017-01-05 ┆ -0.000794 │
 │ 2017-01-06 ┆ 0.0       │
 │ 2017-01-09 ┆ -0.003301 │
 │ 2017-01-10 ┆ -0.003301 │
 │ …          ┆ …         │
 │ 2025-06-23 ┆ -0.015002 │
 │ 2025-06-24 ┆ -0.00412  │
 │ 2025-06-25 ┆ -0.003562 │
 │ 2025-06-26 ┆ 0.0       │
 │ 2025-06-27 ┆ 0.0       │
 └────────────┴───────────┘}

In [391]:
plot_line(calc_max_dd(df, "GLD")["drawdown"], x="Date", y=["drawdown"])

alt.Chart(...)

## 1.3 Quantiles

Report the quantiles of the series. Use `.describe()` for a useful summary.

* Report the 5th quantile scaled by standard deviation.
* How much sampling error is in the mean return? Is this likely to cause much sampling error in the quantile estimation?

In [380]:
series = df["SPY"]
series.describe()

statistic,value
str,f64
"""count""",2132.0
"""null_count""",0.0
"""mean""",0.000607
"""std""",0.011875
"""min""",-0.109424
"""25%""",-0.003709
"""50%""",0.00074
"""75%""",0.006212
"""max""",0.105019


In [382]:
series.quantile(.05) / series.std()

-1.4836575914738328

In [395]:
series.std() / len(series)

5.569693481007723e-06

## 1.4 Comparison

Try repeating 1.1-1.3 for another asset. 

***

# 2. Time Aggregation

Use the data in `../data/risk_etf_data.xlsx`.

Use the price series to calculate **monthly** returns. (You may find `df.resample('M').last()` helpful.

In [408]:
def resample(data: pl.DataFrame, idx: str, var: str, window: str) -> pl.DataFrame:
    """
    Resample daily return data into yearly/quarterly/monthly/weekly
    Args:
        - data: col["Date", ]
        - idx: column name as idx
        - var: column name to consider
        - window: 
            - 1d (1 calendar day)
            - 1w (1 calendar week)
            - 1mo (1 calendar month)
            - 1q (1 calendar quarter)
            - 1y (1 calendar year)
    """
    return data.group_by_dynamic(idx, every=window).agg(
        (pl.col(var) + 1).cum_prod().last() - 1
    )

## 2.1 Bitcoin and Measuring Covariation Risk

People often say that bitcoin has disappointed by being highly correlated to SPY.

Report the correlation between BTC and SPY for both daily and monthly returns.

What do you conclude?

In [411]:
monthly_btc = resample(df, idx="Date", var="BTC", window="1mo")
monthly_spy = resample(df, idx="Date", var="SPY", window="1mo")

In [419]:
monthly_concat = pl.concat([monthly_btc, monthly_spy], how="align")
monthly_concat.select(pl.corr("BTC", "SPY"))

BTC
f64
0.341851


In [422]:
daily_concat = pl.corr(df["BTC"], df["SPY"], eager=True)
daily_concat

BTC
f64
0.250931


## 2.2 Betas

For each series, calculate its beta to SPY.
* Estimate the regression with an intercept (alpha) but no need to report it.

How do these betas compare to the daily return betas seen in the note?

In [424]:
from sklearn.linear_model import LinearRegression

In [459]:
def calc_beta(data: pl.DataFrame, X: str|List[str], y: str):
    """
    Calculate the beta of y on X (y regressed on X)
    """
    y_df = data.select(pl.col(y)).to_numpy().reshape(-1, 1)
    if isinstance(X, str):
        X_df = data.select(pl.col(X)).to_numpy().reshape(-1, 1)
    elif isinstance(X, list):
        X_df = data.select(pl.col(X)).to_numpy()
    
    beta = LinearRegression().fit(X=X_df, y=y_df).coef_[0]
    beta_df = pl.DataFrame({
        "Tickers": X, f"Beta ({y})": beta
    })
    return beta_df

In [461]:
calc_beta(df, X="SPY",y="BTC")

Tickers,Beta (BTC)
str,f64
"""SPY""",0.929589


In [466]:
calc_beta(df, X=["SPY", "VEA"], y="BTC")

Tickers,Beta (BTC)
str,f64
"""SPY""",0.432335
"""VEA""",0.618915


## 2.3 Time Scaling to Higher Moments

Report the skewness and kurtosis for the monthly returns.

How do these compare to the daily skewness and kurtosis measures?

What do you conclude?

***